In [1]:
from dotenv import load_dotenv
from conversationService import get_embedding
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import loggingService
import os
import weaviate

load_dotenv()
logger = loggingService.get_logger()


/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
sentence-transformers/gtr-t5-large


/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/genai/credentials.py:31: DeprecationWarning: The 'api_endpoint' property should not contain any explicit API version(rename it from 'https://workbench-api.res.ibm.com/v1' to just 'https://workbench-api.res.ibm.com')
  warn(


2023-09-28 13:28:34,562 - numexpr.utils - INFO - Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-09-28 13:28:34,562 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
apikey = os.getenv("GEANAI_KEY", None)
class_name = os.getenv("WEVIATE_CLASS", 'LivrosVectorizer')
path = os.getenv("DATA_PATH", "data")
weaviate_url = os.getenv("WEAVIATE_URL", 'http://127.0.0.1:8080')

client = weaviate.Client(
    url=weaviate_url,
)

pages = []
pdf_loader = PyPDFDirectoryLoader(path)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50, separators=["\n\n", "\n", " ", "", "\n \n", "\n \n \n","\n\n\n"])
documents = pdf_loader.load_and_split(text_splitter=text_splitter)
print(len(documents))
print(documents[0])

452
page_content='Dom Casmu rro\nMachado de  Assi s\nCAPÍTULO PRIM EIRO / DO TÍTULO \nUma noite destas, vindo da  cidade para o Engenho N ovo, e ncontrei num  trem da Central um rapaz \naqui do ba irro, que  eu conhe ço de vista e de chapéu. Cum primentou-m e, sentou-se ao pé de mim, \nfalou da  lua e dos ministros, e acabou re citando-m e versos. A viagem era curta, e os versos pode  ser \nque não fos sem inteiramente maus. Sucedeu, poré m, que , como eu estava cansado, fe chei os olhos \ntrês ou qua tro ve zes; tanto bastou pa ra que ele interrom pesse a leitura e metesse os versos no bol so. \n-- Cont inue, disse eu acordando. \n-- Já acabei, murmurou e le. \n-- São muito boni tos. \nVi-lhe fazer um gesto para tirá-los outra vez do bol so, mas não passou do ge sto; estava amuado. N o \ndia seguinte entrou a  dizer de mim nom es feios, e acabou a lcunha ndo-m e Dom Casmurro. O s \nvizinhos , que  não gos tam dos meus hábitos reclusos e calados, deram curso à alcunha , que  afinal' me

In [3]:
def pdf_text_splitter(pdf_text) -> dict:
  retorno = {'content': '', 'source': '', 'page': 0}
  
  retorno['content'] = getattr(pdf_text, 'page_content')
  retorno['souce'] = getattr(pdf_text, 'metadata')['source']
  retorno['page'] = getattr(pdf_text, 'metadata')['page']
  
  return retorno

for doc in documents:
  # logger.debug(pdf_text_splitter(doc))
  # print(pdf_text_splitter(doc))
  pages.append(pdf_text_splitter(doc))
  
print(pages[0])

{'content': 'Dom Casmu rro\nMachado de  Assi s\nCAPÍTULO PRIM EIRO / DO TÍTULO \nUma noite destas, vindo da  cidade para o Engenho N ovo, e ncontrei num  trem da Central um rapaz \naqui do ba irro, que  eu conhe ço de vista e de chapéu. Cum primentou-m e, sentou-se ao pé de mim, \nfalou da  lua e dos ministros, e acabou re citando-m e versos. A viagem era curta, e os versos pode  ser \nque não fos sem inteiramente maus. Sucedeu, poré m, que , como eu estava cansado, fe chei os olhos \ntrês ou qua tro ve zes; tanto bastou pa ra que ele interrom pesse a leitura e metesse os versos no bol so. \n-- Cont inue, disse eu acordando. \n-- Já acabei, murmurou e le. \n-- São muito boni tos. \nVi-lhe fazer um gesto para tirá-los outra vez do bol so, mas não passou do ge sto; estava amuado. N o \ndia seguinte entrou a  dizer de mim nom es feios, e acabou a lcunha ndo-m e Dom Casmurro. O s \nvizinhos , que  não gos tam dos meus hábitos reclusos e calados, deram curso à alcunha , que  afinal', 'sourc

In [4]:
class_document =  {
  "class": "Livros",
  "description": "Dados do documento PDF",
  "moduleConfig": {
    "text2vec-transformers": {
      "poolingStrategy": "masked_mean",
      "vectorizeClassName": False
    }
  },
  "properties": [
    {
      "dataType": [
        "text"
      ],
      "description": "Conteúdo do documento PDF",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "content"
    },
    {
      "dataType": [
        "text"
      ],
      "description": "",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "source"
    },
    {
      "dataType": [
        "text"
      ],
      "description": "",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "page"
    }
  ],
  "vectorizer": "none"
}

class_document2 =  {
  "class": "LivrosVectorizer",
  "description": "Dados do documento PDF",
  "moduleConfig": {
    "text2vec-transformers": {
      "poolingStrategy": "masked_mean",
      "vectorizeClassName": False
    }
  },
  "properties": [
    {
      "dataType": [
        "text"
      ],
      "description": "Conteúdo do documento PDF",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "content"
    },
    {
      "dataType": [
        "text"
      ],
      "description": "",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "source"
    },
    {
      "dataType": [
        "text"
      ],
      "description": "",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "page"
    }
  ],
  "vectorizer": "text2vec-transformers"
}

client.schema.create_class(class_document2)
# client.schema.delete_class('LivrosVectorizer')
# client.schema.get("LivrosVectorizer")
# client.schema.property.create(class_name, {
#     'name': 'exemplar',
#     'dataType': ['text'],
# })


In [ ]:
with client.batch as batch:
  batch.batch_size=10
  i = 0
    
  for page in pages:
    logger.info(f"importing question: {i+1}")
    i = i+1
    
    properties = {
      "content": page["content"],
      "page": str(page["page"]),
      "source": page["source"],
    }
    
    # vector = get_embedding(page["content"])

    # client.batch.add_data_object(properties, 'LivrosVectorizer', vector=vector)
    client.batch.add_data_object(properties, 'LivrosVectorizer')

In [ ]:
result = (client.query
.get('LivrosVectorizer', ["content", "source", "page"])
.with_additional(["certainty", "distance"]) # note that certainty is only supported if distance==cosine
.with_near_vector({
  "vector": get_embedding('o que significa casmurro'),
  "certainty": 0.8
})
.with_limit(4)
.do()
)

print(result['data']['Get'][class_name])

retorno = ''
if len(result['data']['Get'][class_name]) > 0:
  retorno = result['data']['Get'][class_name][0]['content']

  for contexto in result['data']['Get'][class_name][1:]:
    retorno += f"\n{contexto['content']}"

logger.info(retorno)
